In [2]:
import joblib
import pandas as pd

In [3]:
sdf = pd.read_csv('dataset/sdf_dataset.csv')

In [4]:
# Load the CountVectorizer object
count = joblib.load('model/count_vectorizer.pkl')

# Load the cosine similarity matrix
cosine_sim = joblib.load('model/cosine_sim_matrix.pkl')

# Load the trained SVD model
model = joblib.load('model/svd_recommender_model.pkl')

# Load the id_map and indices_map
id_map = joblib.load('model/id_map.pkl')
indices_map = joblib.load('model/indices_map.pkl')
indices = joblib.load('model/indices_series.pkl')

In [5]:
def hybrid_recommendations(userId, title):
    title = str(title).title()
    idx = indices[title]
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = sdf.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: model.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [6]:
hybrid_recommendations(6, 'toy story')

,title,vote_count,vote_average,year,id,est
3817,"Monsters, Inc.",6150.0,7.5,2001,585,4.267420
2514,Toy Story 2,3914.0,7.3,1999,863,3.919054
8555,The Lego Movie,3127.0,7.5,2014,137106,3.828403
7589,Toy Story 3,4710.0,7.6,2010,10193,3.822331
7133,A Matter of Loaf and Death,120.0,7.2,2008,14447,3.693756
1875,A Bug's Life,2379.0,6.8,1998,9487,3.600464
4317,"The Looney, Looney, Looney Bugs Bunny Movie",13.0,6.6,1981,41394,3.325841
7218,Kung Fu Panda: Secrets of the Furious Five,80.0,6.5,2008,15854,3.310732
1824,Antz,1320.0,6.0,1998,8916,3.193978
6354,Luxo Jr.,148.0,7.1,1986,13925,3.134774
